In [39]:
import pandas as pd
import os

from datalayer import read_csv_sia, read_sia_model, _merge_by_year_and_month
import utils
import feature_engineering
import external_data

In [41]:
data = read_csv_sia('../data/Mama Radioterapia SIA-SUS.csv', 'radioterapia')

data = data[data['AP_TPAPAC']==1] # removes data that are not from the first authorization

data = data[data['AR_DTIDEN'] >= pd.to_datetime('2014-01-01')].copy() # filter date: date >= 2014-01-01

data = feature_engineering.transform_cep_in_feature(data, ['AP_CEPPCN'])

data = external_data.get_municipio_info(data, ['AP_MUNPCN', 'AP_UFMUN'])
data = external_data.get_municipio_info_atlas(data, ['AP_MUNPCN'])

data = external_data.get_cep_info(data, ['AP_CEPPCN'])
data = external_data.get_cnes_loc(data, ['AP_CODUNI']) 
    
data = utils.create_year_month_date(data, ['AR_DTIDEN'])
data = utils.create_year_date(data, ['AR_DTIDEN'])

data = external_data.get_orcamento_publico(data, ['AP_MUNPCN'], 'AR_DTIDEN_YEAR') 

data = _merge_by_year_and_month(data, ESTABELECIMENTO_FILES, 'estabelecimento')

data['DISTANCE_HOSPITAL'] = data.apply(lambda x: utils.calc_distance_lat_long(x['AP_CEPPCN_LATITUDE'],
                                                                                x['AP_CEPPCN_LONGITUDE'],
                                                                                x['AP_CODUNI_LATITUDE'],
                                                                                x['AP_CODUNI_LONGITUDE']), 1)

    
data = data[['AP_CODUNI', 'AP_CODUNI_LATITUDE' , 'AP_CODUNI_LONGITUDE', 'AP_UFMUN']]

b'Skipping line 100354: expected 74 fields, saw 87\n'


In [42]:
review = pd.read_csv('data/reviews.csv', sep=';')
review['Nota'] = review['Nota'].apply(pd.to_numeric, errors='coerce')
review.columns = [col.upper() for col in review.columns]
    
columns = ['AP_CODUNI']
review_col = review.copy()
review_col.columns = ['{0}_{1}'.format(col, x) for x in review_col]
review_col[col] = review_col['{0}_{1}'.format(col, 'AP_CODUNI')]

data = data.merge(review_col, how='left', on=col)
data = data.drop('{0}_{1}'.format(col, 'AP_CODUNI'), 1)

In [43]:
data.head()

,AP_CODUNI,AP_CODUNI_LATITUDE,AP_CODUNI_LONGITUDE,AP_UFMUN,AP_CODUNI_NOTA,AP_CODUNI_QT
0,2001586,-9.9774,-67.80585,120040,3.8,16.0
1,2001586,-9.9774,-67.80585,120040,3.8,16.0
2,2001586,-9.9774,-67.80585,120040,3.8,16.0
3,2001586,-9.9774,-67.80585,120040,3.8,16.0
4,2001586,-9.9774,-67.80585,120040,3.8,16.0


In [44]:
cnes_df = pd.read_csv('../data/espelho cnes nome fantasia.csv', sep=';', encoding='latin1')
cnes_df = cnes_df.rename(columns={'cnes': 'AP_CODUNI'})

In [45]:
left_hosp = data[data['AP_CODUNI_NOTA'].isna()]
left_hosp = left_hosp[['AP_CODUNI', 'AP_CODUNI_LATITUDE' , 'AP_CODUNI_LONGITUDE', 'AP_UFMUN']]

In [ ]:
data_model = data_model.merge(cnes_df, on='AP_CODUNI', how='left')

In [37]:
left_hosp = data.groupby('AP_CODUNI').count()

In [38]:
left_hosp.iloc[:15,:]

,AP_CODUNI_LATITUDE,AP_CODUNI_LONGITUDE,AP_UFMUN,AP_CODUNI_NOTA,AP_CODUNI_QT
AP_CODUNI,,,,,
434,745,745,745,745,745
582,1576,1576,1576,1576,1576
1023,1284,1284,1284,0,1284
2283,163,163,163,163,163
3786,3903,3903,3903,3903,3903
3808,544,544,544,544,544
3832,543,543,543,543,543
4251,40,40,40,0,40
8753,608,608,608,608,608
